This module provides utility functions to save complex bidder demand dictionaries as JSON files.

In [5]:
import pandas as pd
import numpy as np
import json

def save_to_json(bidder_demand_dict, filename):
    """
    Save bidder_demand_dict to a JSON file, ensuring type compatibility.

    Parameters:
    - bidder_demand_dict: dict, a dictionary with bidders as keys containing quantity and price demand
    - filename: str, the name of the output file (e.g., "bidder_demand_dict.json")
    """
    # Create a converted dictionary to ensure all data types are JSON serializable
    def convert_to_serializable(obj):
        """ Recursively convert non-JSON-serializable types to native Python types. """
        if isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(convert_to_serializable(item) for item in obj)
        elif isinstance(obj, (int, float, str)):
            return obj
        else:
            # Attempt to convert to string as a fallback
            return str(obj)

    # Convert the dictionary to a serializable format
    serializable_dict = convert_to_serializable(bidder_demand_dict)
    
    # Write the converted data to a JSON file
    try:
        with open(filename, 'w') as f:
            json.dump(serializable_dict, f, indent=4)
        print(f"Data successfully saved to file: {filename}")
    except (TypeError, ValueError) as e:
        print(f"Failed to save file: {e}")


This module provides functions to organize and analyze bid panel data by bidder.
The `group_bids_by_bidder` function groups the bidding data into separate DataFrames for each bidder, while `get_bidder_markets` extracts the list of unique markets (defined by `(market_number, category)`) each bidder has participated in, returning the results as a dictionary.

In [6]:
def group_bids_by_bidder(bid_data):
    """
    Group bid panel data by the 'bidder' column and return a list of DataFrames, 
    each corresponding to a single bidder.

    Parameters:
    - bid_data: pandas DataFrame, containing the bid panel data

    Returns:
    - list: a list of DataFrames, each containing data for one bidder
    """
    # Group by 'bidder' column
    bidder_groups = [group for _, group in bid_data.groupby('bidder')]
    return bidder_groups

def get_bidder_markets(bidder_groups):
    """
    Identify all markets each bidder has placed bids in across all rounds,
    and return the results as a dictionary.

    Parameters:
    - bidder_groups: list, a list of DataFrames, each containing data for a single bidder

    Returns:
    - dict: a dictionary where each key is a bidder, and the value is a list of (market_number, category) tuples
    """
    bidder_market_dict = {}

    for bidder_data in bidder_groups:
        # Get the bidder's ID
        bidder_id = bidder_data['bidder'].iloc[0]
        
        # Get a list of unique (market_number, category) tuples the bidder has bid in
        markets = list(
            bidder_data[['market_number', 'category']]
            .drop_duplicates()
            .apply(tuple, axis=1)
        )
        
        # Store in dictionary
        bidder_market_dict[bidder_id] = markets

    return bidder_market_dict



This function computes each bidder’s **round-level demand vectors**, including quantity, price, and rivals’ quantity, for all the markets they participated in.

It takes grouped bidder data and a bidder-to-market mapping as input, and returns a dictionary mapping each bidder to a list of tuples—each tuple representing a round with its corresponding demand vectors.

In [7]:
def calculate_round_demand_by_bidder(bidder_groups, bidder_market_dict):
    """
    Calculate each bidder's demand vectors (quantity, price, rivals' quantity) 
    for each round, along with the round number.

    Parameters:
    - bidder_groups: list, a list of DataFrames, each containing data for one bidder
    - bidder_market_dict: dict, a dictionary mapping each bidder to their list of (market_number, category) tuples

    Returns:
    - dict: a dictionary mapping each bidder to a list of tuples, 
            each containing (round_id, quantity_vector, price_vector, rivals_quantity_vector)
    """
    bidder_demand_dict = {}

    for bidder_data in bidder_groups:
        bidder_id = bidder_data['bidder'].iloc[0]
        markets = bidder_market_dict[bidder_id]
        round_demand_list = []

        # Group by round to extract demand vectors per round
        for round_id, round_data in bidder_data.groupby('round'):
            quantity_vector = []
            price_vector = []
            rivals_quantity_vector = []

            # Loop through all markets the bidder has participated in
            for market in markets:
                market_data = round_data[
                    (round_data['market_number'] == market[0]) &
                    (round_data['category'] == market[1])
                ]

                if not market_data.empty:
                    quantity = int(market_data['quantity'].iloc[0])
                    price = int(market_data['bid_amount'].iloc[0])
                    rivals_quantity = (
                        0 if np.isnan(market_data['aggregate_rivals_quantity'].iloc[0])
                        else int(market_data['aggregate_rivals_quantity'].iloc[0])
                    )
                else:
                    quantity = 0
                    price = 0
                    rivals_quantity = 0

                quantity_vector.append(quantity)
                price_vector.append(price)
                rivals_quantity_vector.append(rivals_quantity)

            # Append the demand vector tuple for this round
            round_demand_list.append((
                round_id, quantity_vector, price_vector, rivals_quantity_vector
            ))

        bidder_demand_dict[bidder_id] = round_demand_list

    return bidder_demand_dict


This script reads preprocessed bidding data and computes **per-bidder, per-round demand vectors** for all participating markets. It performs the following steps:

1. **Loads** the bidding panel data from CSV.
2. **Groups** bids by each bidder using `group_bids_by_bidder`.
3. **Identifies** all markets each bidder participated in via `get_bidder_markets`.
4. **Computes** round-level demand vectors—quantities, prices, and rivals’ quantities—using `calculate_round_demand_by_bidder`.

In [8]:
bids_data = pd.read_csv("../data/processed data/clock_bids_with_rival_qty.csv")  # 示例路径

# Apply the function and print the number of bidder groups
bidder_groups = group_bids_by_bidder(bids_data)
print(len(bidder_groups))

bidder_market_dict = get_bidder_markets(bidder_groups)
print(bidder_market_dict['T-Mobile License LLC'])
print(len(bidder_market_dict['T-Mobile License LLC']))

bidder_demand_dict = calculate_round_demand_by_bidder(bidder_groups, bidder_market_dict)
print(bidder_demand_dict['T-Mobile License LLC'][4])
print(len(bidder_demand_dict['T-Mobile License LLC']))

save_to_json(bidder_demand_dict, "../data/processed data/bidder_demand_dict.json")

/var/folders/_x/43n5hx2d6hqf1v6v65qbln8c0000gn/T/ipykernel_27548/3618610456.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bids_data = pd.read_csv("../data/processed data/clock_bids_with_rival_qty.csv")  # 示例路径


44
[('PEA001', 'A'), ('PEA001', 'BC'), ('PEA002', 'A'), ('PEA002', 'BC'), ('PEA003', 'A'), ('PEA003', 'BC'), ('PEA004', 'A'), ('PEA004', 'BC'), ('PEA005', 'ABC'), ('PEA006', 'A'), ('PEA006', 'BC'), ('PEA007', 'A'), ('PEA007', 'BC'), ('PEA008', 'A'), ('PEA008', 'BC'), ('PEA009', 'A'), ('PEA009', 'BC'), ('PEA010', 'A'), ('PEA010', 'BC'), ('PEA011', 'ABC'), ('PEA012', 'A'), ('PEA012', 'BC'), ('PEA013', 'A'), ('PEA013', 'BC'), ('PEA014', 'A'), ('PEA014', 'BC'), ('PEA015', 'A'), ('PEA015', 'BC'), ('PEA016', 'A'), ('PEA016', 'BC'), ('PEA017', 'A'), ('PEA017', 'BC'), ('PEA018', 'A'), ('PEA018', 'BC'), ('PEA020', 'ABC'), ('PEA021', 'A'), ('PEA021', 'BC'), ('PEA024', 'A'), ('PEA024', 'BC'), ('PEA026', 'A'), ('PEA026', 'BC'), ('PEA027', 'A'), ('PEA027', 'BC'), ('PEA028', 'A'), ('PEA028', 'BC'), ('PEA029', 'A'), ('PEA029', 'BC'), ('PEA030', 'A'), ('PEA030', 'BC'), ('PEA031', 'A'), ('PEA031', 'BC'), ('PEA034', 'A'), ('PEA034', 'BC'), ('PEA035', 'A'), ('PEA035', 'BC'), ('PEA036', 'A'), ('PEA036', '